### Import libraries

#### Run the same model and add the correctly labeled tweets based on the prediction on the first model
- We used KNN to predict the first model. Out of the 50 test data, 32 were correct.

In [1]:
import numpy as np
import pandas as pd
import scipy as sp   
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.cross_validation import train_test_split #how does this work?
from sklearn.model_selection import train_test_split as tts

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
neighbors = 5
from sklearn.naive_bayes import MultinomialNB
from nltk.classify import NaiveBayesClassifier  #how does this work?

from sklearn.model_selection import cross_val_score as cvs

/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
kanye2 = pd.read_csv('kwest.csv')
kanye2.head()

,text,label
0,RT @WSHHRAP: Rappers with the most Billboard H...,1.0
1,"Kanye and Cudi merge, together they rule as Kanye",1.0
2,RT @RappersSaid: Rappers with the most Billboa...,1.0
3,RT @c_drew_: Kanye shaped like he played barit...,1.0
4,RT @Bubbabue: @kanyewest Your album #ye is an ...,1.0


In [7]:
kanye2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2064 entries, 0 to 2063
Data columns (total 2 columns):
text     2064 non-null object
label    370 non-null float64
dtypes: float64(1), object(1)
memory usage: 32.3+ KB


In [8]:
df = kanye2.sort_values(by = 'label', ascending=True)
df1 = df.reset_index()
df1[369:].head()

,index,text,label
369,0,RT @WSHHRAP: Rappers with the most Billboard H...,1.0
370,370,I need someone who can edit to put Bebe Zahara...,NaN
371,371,When???!!!! Lmao yall b On SuperXtra LMMMAoo h...,NaN
372,372,RT @new_branches: Drake wrote the hook for Kan...,NaN
373,373,@BHOPDaMaestro by Lil Baby ft. Drake &amp; Kan...,NaN


In [9]:
kanye2train = df1[(df1['label'] ==0)|(df1['label']==1)]
kanye2train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 370 entries, 0 to 369
Data columns (total 3 columns):
index    370 non-null int64
text     370 non-null object
label    370 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 11.6+ KB


In [10]:
kanye2train.head()

,index,text,label
0,184,"RT @RealCandaceO: The mob is loud, but the maj...",0.0
1,204,@kanyewest I miss the old Kanye. We want pre-nap!,0.0
2,203,RT @MattsMentions: “I didn’t listen to Kanye’s...,0.0
3,202,Honestly I fucks with Kanye’s style. Rich peop...,0.0
4,201,"@GOPChairwoman @realDonaldTrump ""I WILL NEVER ...",0.0


In [53]:
kanye2test = df1[df1['label'].isnull()][:75]
kanye2test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 370 to 444
Data columns (total 3 columns):
index    75 non-null int64
text     75 non-null object
label    0 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.3+ KB


In [54]:
kanye2test.head()

,index,text,label
370,370,I need someone who can edit to put Bebe Zahara...,NaN
371,371,When???!!!! Lmao yall b On SuperXtra LMMMAoo h...,NaN
372,372,RT @new_branches: Drake wrote the hook for Kan...,NaN
373,373,@BHOPDaMaestro by Lil Baby ft. Drake &amp; Kan...,NaN
374,374,I’ve never heard Kanye West talk until celebri...,NaN


In [55]:
text = kanye2train['text']
print(type(text))
text.head()

<class 'pandas.core.series.Series'>


0    RT @RealCandaceO: The mob is loud, but the maj...
1    @kanyewest I miss the old Kanye. We want pre-nap!
2    RT @MattsMentions: “I didn’t listen to Kanye’s...
3    Honestly I fucks with Kanye’s style. Rich peop...
4    @GOPChairwoman @realDonaldTrump "I WILL NEVER ...
Name: text, dtype: object

In [56]:
toadd = kanye2test['text']
toadd

370    I need someone who can edit to put Bebe Zahara...
371    When???!!!! Lmao yall b On SuperXtra LMMMAoo h...
372    RT @new_branches: Drake wrote the hook for Kan...
373    @BHOPDaMaestro by Lil Baby ft. Drake &amp; Kan...
374    I’ve never heard Kanye West talk until celebri...
375        Someone stop this guy https://t.co/bEXdtJViQR
376    RT @SnakesPod: Check out the latest from #snak...
377    RT @Chrisshane8: I created an almost real Scot...
378    RT @SirCoach: So its gonna be all downhill aft...
379    RT @JakellDaGOAT: But if he actually commit su...
380    kanye got loaned gza's mic for reborn. sounds ...
381    (((FREE download))) "TRUE LIES" a collection o...
382                 Drake’s &amp; Kanye’s twins &gt;&gt;
383    @Phillyronan Kanye producing the album as well...
384    RT @LaurenWern: Why? Is Kanye not available? h...
385    RT @TeamKanyeDaily: @Genius You guys are so me...
386    Y'all ain't file for bankruptcy yet? https://t...
387       Welcome to the Ye fam

### Let's try TF-IDF

In [57]:
tfidf = text.append(toadd, ignore_index = True)
tfidf.head()

0    RT @RealCandaceO: The mob is loud, but the maj...
1    @kanyewest I miss the old Kanye. We want pre-nap!
2    RT @MattsMentions: “I didn’t listen to Kanye’s...
3    Honestly I fucks with Kanye’s style. Rich peop...
4    @GOPChairwoman @realDonaldTrump "I WILL NEVER ...
Name: text, dtype: object

In [58]:
vect = TfidfVectorizer(stop_words='english')

In [59]:
tf = pd.DataFrame(vect.fit_transform(tfidf).toarray(), columns=vect.get_feature_names())

In [60]:
tf.head()

,0iou2pgxch,0jujild7px,100,1063atl,10s,11,12,135,148lqlf5ao,16,...,yo,yourmomresists,youtube,ywltohwavw,yxotobksfb,zahara,zmh9qhh6xk,znu3q5mi58,zth5gym7xn,zuqwdrzfdd
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.45018,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
tf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Columns: 1093 entries, 0iou2pgxch to zuqwdrzfdd
dtypes: float64(1093)
memory usage: 3.7 MB


In [62]:
reset = kanye2train['label'].reset_index()
del reset ['index']
reset.head()

,label
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [63]:
kanye = pd.concat([tf,reset], axis = 1,ignore_index= False)
kanye['label'][369:].head()

369    1.0
370    NaN
371    NaN
372    NaN
373    NaN
Name: label, dtype: float64

In [64]:
edittrain = kanye[:370]
edittest = kanye[370:]
droplabel = edittest.drop('label', axis = 1)

In [65]:
edittrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Columns: 1094 entries, 0iou2pgxch to label
dtypes: float64(1094)
memory usage: 3.1 MB


In [66]:
edittest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 370 to 444
Columns: 1094 entries, 0iou2pgxch to label
dtypes: float64(1094)
memory usage: 641.1 KB


In [67]:
x = edittrain.drop('label', axis = 1)
y = edittrain['label']

In [68]:
xtrain, xtest, ytrain, ytest = tts(x,y, random_state = 0)

In [69]:
l = LogisticRegression()
d = DecisionTreeClassifier()
r = RandomForestClassifier()
g = GradientBoostingClassifier()
k = KNeighborsClassifier()
nb = MultinomialNB()
vc = VotingClassifier(estimators = [('l',l),('d',d),('r',r),('g',g),('k',k),('nb',nb)],
                      voting = 'hard', n_jobs = 5)

In [70]:
def classifier(ye):
    print(ye)
    clf = ye.fit(xtrain,ytrain)
    clf.predict(droplabel)
    print('Accuracy of train: ', ye.score(xtrain,ytrain))
    print('Accuracy of test: ', ye.score(xtest,ytest))
    score = cvs(ye,xtest,ytest,cv=5, scoring='accuracy')
    print('cross val score: ', score.mean())

In [71]:
classifier(LogisticRegression())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Accuracy of train:  0.9711191335740073
Accuracy of test:  0.8709677419354839
cross val score:  0.8070175438596492


In [72]:
classifier(DecisionTreeClassifier())

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
Accuracy of train:  1.0
Accuracy of test:  0.8387096774193549
cross val score:  0.7631578947368421


In [73]:
classifier(RandomForestClassifier())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Accuracy of train:  0.9963898916967509
Accuracy of test:  0.8709677419354839
cross val score:  0.7637426900584796


In [74]:
classifier(GradientBoostingClassifier())

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)
Accuracy of train:  1.0
Accuracy of test:  0.8602150537634409
cross val score:  0.7526315789473685


In [75]:
classifier(KNeighborsClassifier())

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
Accuracy of train:  0.924187725631769
Accuracy of test:  0.9032258064516129
cross val score:  0.828654970760234


In [76]:
classifier(MultinomialNB())

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
Accuracy of train:  0.9855595667870036
Accuracy of test:  0.8709677419354839
cross val score:  0.8502923976608188


In [77]:
classifier(vc)

VotingClassifier(estimators=[('l', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('d', DecisionT...
           weights='uniform')), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         flatten_transform=None, n_jobs=5, voting='hard', weights=None)


/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Accuracy of train:  1.0
Accuracy of test:  0.8602150537634409


/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site

cross val score:  0.7748538011695907


/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### Extracting Ypred

In [78]:
knn = kn.fit(xtrain,ytrain)
ypred = kn.predict(droplabel)
ypred

array([1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1.])

In [79]:
len(ypred)

75

In [80]:
len(toadd)

75

In [81]:
results = pd.DataFrame(columns=['text','label'])
results['text'] = toadd
results['label'] = ypred
results.to_csv('kanye.csv')